In [154]:
import pandas as pd
import pickle
import gzip
import os
import pandas as pd
from dbfread import DBF
import folium
import base64
from folium import plugins
print(folium.__version__)

0.8.3


In [2]:
from google.colab import files
uploaded = files.upload()

Saving seoul_populartime.pickle to seoul_populartime.pickle
Saving subway_xy.pickle to subway_xy.pickle


In [3]:
#장소랑 지하철 좌표 불러오기
with gzip.open('seoul_populartime.pickle','rb') as f:
    data = pickle.load(f)
with gzip.open('subway_xy.pickle','rb') as f:
    subway = pickle.load(f)

In [16]:
# 장소 하나의 데이터프레임으로 만들기
places = list(data.keys())
print(places)
df = pd.DataFrame()
for i in places:
    df1 = pd.DataFrame(data[i])
    df1['places'] = i
    df = pd.concat([df,df1])
df = df.reset_index(drop=True)

['amusement_park', 'aquarium', 'art_gallery', 'bakery', 'bank', 'bar', 'book_store', 'bowling_alley', 'cafe', 'clothing_store', 'department_store', 'doctor', 'drugstore', 'hospital', 'library', 'local_government_office', 'meal_takeaway', 'movie_theater', 'museum', 'night_club', 'park', 'pharmacy', 'post_office', 'restaurant', 'shopping_mall', 'spa', 'store', 'tourist_attraction', 'zoo']


In [17]:
print(len(df),len(df.columns))

21531 12


In [23]:
place_dic = {'amusement_park' : '관광/문화/놀거리' , 'aquarium' : '관광/문화/놀거리' ,
             'art_gallery' : '관광/문화/놀거리' , 'book_store' : '관광/문화/놀거리',
             'bowling_alley' : '관광/문화/놀거리', 'library' : '관광/문화/놀거리',
             'movie_theater' : '관광/문화/놀거리', 'museum' : '관광/문화/놀거리',
             'doctor' : '의료','drugstore' : '의료',
             'hospital' : '의료','pharmacy' : '의료',
             'bar' : '맛집','meal_takeaway' : '맛집','restaurant' : '맛집',
             'bakery' : '카페/빵집', 'cafe' : '카페/빵집',
             'clothing_store' : '쇼핑/마트', 'department_store' : '쇼핑/마트',
             'shopping_mall' : '쇼핑/마트','store' : '쇼핑/마트',
             'bank' : '은행/공공기관', 'local_government_office' : '은행/공공기관','post_office' : '은행/공공기관',
            }

In [24]:
# X, Y 좌표로 데이터 프레임 빼기
def preprocessing_data(df):
    df['lat'] = [df['coordinates'][i]['lat'] for i in range(len(df))]
    df['lng'] = [df['coordinates'][i]['lng'] for i in range(len(df))]
    df['category'] = df['places'].map(place_dic)
    return df
df = preprocessing_data(df)

In [257]:
class DF:
    def __init__(self, df):
        self.df = df
        
        
    def listup(self, station, category = 'all', k=0.01):
        input_coordinates = subway[station]
        df['lat_dis'] = abs(df['lat'] - input_coordinates[0])
        df['lng_dis'] = abs(df['lng'] - input_coordinates[1])
        self.df_sta_cat = df.query("lat_dis < {} and lng_dis < {}".format(k,k))
        
        if category == 'all':
            return self.df_sta_cat
        else:
            self.df_sta_cat = self.df_sta_cat[self.df_sta_cat['category']== category]
        return self.df_sta_cat

        
    def timeline(self, time = (0,24), week='Monday'):#time=(시작시간,끝시간)
        time_dic={}
        temp = []
        
        for i in self.df_sta_cat['populartimes']:
            for j in i:
                time_dic[j['name']] = j['data']
            temp.append(pd.DataFrame(time_dic)[week][time[0]:time[1]].values)
        self.df_sta_cat['인구수'] = temp
        self.df_sta_cat = self.df_sta_cat.reset_index(drop=True)
        return self.df_sta_cat[['name','address','인구수','lat','lng']]
    
    def map_list(self, station, data):
      m = folium.Map(
          location = subway[station],
          zoom_start = 15
      )
      # 이거 특정기준이 잡히면 그냥 하나의 컬럼을 만들어서 color안에 넣을 수 있게 줄일 예정
      for i in t.index:
        if t['인구수'].iloc[i] > 75:
          marker = folium.CircleMarker([t['lat'].iloc[i],t['lng'].iloc[i]],color='red')
          marker.add_to(m)
        elif t['인구수'].iloc[i] > 50:
          marker = folium.CircleMarker([t['lat'].iloc[i],t['lng'].iloc[i]],color='yellow')
          marker.add_to(m)
        elif t['인구수'].iloc[i] > 25:
          marker = folium.CircleMarker([t['lat'].iloc[i],t['lng'].iloc[i]],color='green')
          marker.add_to(m)
        else:
          marker = folium.CircleMarker([t['lat'].iloc[i],t['lng'].iloc[i]],color='blue')
          marker.add_to(m)
      return m


In [258]:
dd = DF(df)

In [259]:
dd.listup('강남', '맛집') ##listup으로 역이랑 카테고리 정함

,id,name,address,types,coordinates,rating,rating_n,current_popularity,populartimes,time_spent,places,time_wait,lat,lng,lat_dis,lng_dis,category
1658,ChIJyQVg4k2hfDUReMelAIyw3WA,블루문 믹스앤몰트,"1640-13 Seocho 1(il)-dong, Seocho-gu","[bar, point_of_interest, establishment]","{'lat': 37.49405180000001, 'lng': 127.0188453}",4.5,237.0,NaN,"[{'name': 'Monday', 'data': [8, 3, 0, 0, 0, 0,...","[90, 150]",bar,NaN,37.494052,127.018845,0.003123,0.009081,맛집
1662,ChIJ1_TmpEKhfDUR9cJWDczNEZY,"BAR 11 CLUB (바11클럽,바일레븐)","1330-15 Seocho-dong, Seocho-gu","[bar, point_of_interest, establishment]","{'lat': 37.4935841, 'lng': 127.0285937}",3.9,30.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[120, 240]",bar,NaN,37.493584,127.028594,0.003591,0.000668,맛집
1663,ChIJAQ_zjVqhfDUR8lE4oJwP8xE,Tumbler Beer,"38 Seocho-daero 78-gil, Seocho-gu","[bar, point_of_interest, establishment]","{'lat': 37.4945167, 'lng': 127.0282167}",3.5,64.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[90, 150]",bar,NaN,37.494517,127.028217,0.002658,0.000291,맛집
1664,ChIJFbcB-VqhfDURmlU8LpYU-yc,이태원천상 강남역점,"서초 대로 74 길 23, Gangnam-gu","[bar, restaurant, food, point_of_interest, est...","{'lat': 37.495325, 'lng': 127.027105}",3.7,22.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[60, 180]",bar,NaN,37.495325,127.027105,0.001850,0.000821,맛집
1666,ChIJpV8m61ahfDURJDFhRfoC7cw,Just a Glass of Wine,"837-24 Yeoksam-dong, Gangnam-gu","[bar, point_of_interest, establishment]","{'lat': 37.49053869999999, 'lng': 127.0324575}",4.2,33.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[90, 180]",bar,NaN,37.490539,127.032458,0.006636,0.004532,맛집
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14307,ChIJewP4jlWhfDUREKUZe7Fd4u0,원가네감자탕,"60 Yeoksam-ro 17-gil, Gangnam-gu","[restaurant, food, point_of_interest, establis...","{'lat': 37.4985569, 'lng': 127.0364708}",3.9,15.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[45, 45]",restaurant,NaN,37.498557,127.036471,0.001382,0.008545,맛집
14308,ChIJhbeKjlWhfDURXsuK77cZkgg,돈우애락,역삼동 738 강남구 서울특별시 KR,"[restaurant, food, point_of_interest, establis...","{'lat': 37.498493, 'lng': 127.036424}",4.5,36.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...","[60, 150]",restaurant,NaN,37.498493,127.036424,0.001318,0.008498,맛집
14309,ChIJV3ExelWhfDURlt6iphZiZhA,푸른바다세꼬시,"3 Nonhyeon-ro 81-gil, Gangnam-gu","[restaurant, food, point_of_interest, establis...","{'lat': 37.4980443, 'lng': 127.0376879}",4.2,121.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",NaN,restaurant,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",37.498044,127.037688,0.000869,0.009762,맛집
14312,ChIJs7Xe8v6jfDUReYxFpkyQe2w,Chicken & Beer,"509 Nonhyeon-ro, Gangnam-gu","[bar, restaurant, food, point_of_interest, est...","{'lat': 37.5018927, 'lng': 127.0360738}",3.9,31.0,NaN,"[{'name': 'Monday', 'data': [0, 0, 0, 0, 0, 0,...",NaN,restaurant,NaN,37.501893,127.036074,0.004718,0.008148,맛집


In [260]:
dd.listup('강남', '맛집') ##listup으로 역이랑 카테고리 정함
# dd.timeline(time=(16,17), week='Monday') ##특정시간대 인구수 표시
t = dd.timeline(time=(22,23), week='Friday') ##특정시간대 인구수 표시
dd.map_list('강남',t)